<a href="https://colab.research.google.com/github/kkrusere/youTube-comments-Analyzer/blob/main/LLM_fine-tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
import os

#mounting google drive
drive.mount('/content/drive')

########################################

#changing the working directory
os.chdir("/content/drive/MyDrive/EV NLP Data")

!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/EV NLP Data


In [7]:
def load_comments_from_json(filename = 'youtube_comments.json'):
    with open(filename, 'r') as json_file:
        comments = json.load(json_file)
    return comments

In [12]:
import json
import pandas as pd
comments = load_comments_from_json()
df = pd.DataFrame(comments)
df.head()

,comment_text,like_count,reply_count
0,A major obstacle to EV adoption that is always...,6K,507 replies
1,A major obstacle to EV adoption that is always...,6K,507 replies
2,"Prices are too high, and dealerships keep addi...",3.9K,216 replies
3,The government isn’t fast enough to patch poth...,89,6 replies
4,We have the coldest winters in many years here...,34,1 reply


In [13]:
df.to_csv('youtube_comments.csv', index=False)

In [2]:
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !kaggle datasets download -d widhiwinata/twitter-and-reddit-sentiment-analysis
# !unzip twitter-and-reddit-sentiment-analysis.zip
# !rm twitter-and-reddit-sentiment-analysis.zip

In [3]:
!pip install nltk

We are going to do a semi supervised lable propergation

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
reddit_data = pd.read_csv("Reddit_Data.csv")
reddit_data.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [6]:
reddit_data.shape

(37249, 2)

In [7]:
reddit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37249 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37249 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 582.1+ KB


In [8]:
# Dropping the null from the `clean_comment` column
reddit_data.dropna(axis=0, subset=['clean_comment'], inplace=True)
reddit_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37149 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37149 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 870.7+ KB


In [9]:
reddit_data.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


We are going to create 2 more columns one that is lemmentized and the other that is stemmed

In [10]:
# Initialize the Porter Stemmer
stemmer = PorterStemmer()

# Apply stemming to the comments
reddit_data['clean_comment_stemmed'] = reddit_data['clean_comment'].apply(lambda x: [stemmer.stem(word) for word in x.split()])
reddit_data['clean_comment_stemmed_str'] = reddit_data['clean_comment_stemmed'].apply(' '.join)



In [11]:
reddit_data.head()

,clean_comment,category,clean_comment_stemmed,clean_comment_stemmed_str
0,family mormon have never tried explain them t...,1,"[famili, mormon, have, never, tri, explain, th...",famili mormon have never tri explain them they...
1,buddhism has very much lot compatible with chr...,1,"[buddhism, ha, veri, much, lot, compat, with, ...",buddhism ha veri much lot compat with christia...
2,seriously don say thing first all they won get...,-1,"[serious, don, say, thing, first, all, they, w...",serious don say thing first all they won get i...
3,what you have learned yours and only yours wha...,0,"[what, you, have, learn, your, and, onli, your...",what you have learn your and onli your what yo...
4,for your own benefit you may want read living ...,1,"[for, your, own, benefit, you, may, want, read...",for your own benefit you may want read live bu...


In [12]:
# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()
# Apply lemmatization to the comments
reddit_data['clean_comment_lemmatized'] = reddit_data['clean_comment'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x.split()])
reddit_data['clean_comment_lemmatized_str'] = reddit_data['clean_comment_lemmatized'].apply(' '.join)



In [13]:
reddit_data.head()

,clean_comment,category,clean_comment_stemmed,clean_comment_stemmed_str,clean_comment_lemmatized,clean_comment_lemmatized_str
0,family mormon have never tried explain them t...,1,"[famili, mormon, have, never, tri, explain, th...",famili mormon have never tri explain them they...,"[family, mormon, have, never, tried, explain, ...",family mormon have never tried explain them th...
1,buddhism has very much lot compatible with chr...,1,"[buddhism, ha, veri, much, lot, compat, with, ...",buddhism ha veri much lot compat with christia...,"[buddhism, ha, very, much, lot, compatible, wi...",buddhism ha very much lot compatible with chri...
2,seriously don say thing first all they won get...,-1,"[serious, don, say, thing, first, all, they, w...",serious don say thing first all they won get i...,"[seriously, don, say, thing, first, all, they,...",seriously don say thing first all they won get...
3,what you have learned yours and only yours wha...,0,"[what, you, have, learn, your, and, onli, your...",what you have learn your and onli your what yo...,"[what, you, have, learned, yours, and, only, y...",what you have learned yours and only yours wha...
4,for your own benefit you may want read living ...,1,"[for, your, own, benefit, you, may, want, read...",for your own benefit you may want read live bu...,"[for, your, own, benefit, you, may, want, read...",for your own benefit you may want read living ...


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [15]:
def train_and_evaluate_logisticR(X, y = reddit_data['category']):
  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

  # Train a logistic regression model
  logistic_model = LogisticRegression()
  logistic_model.fit(X_train, y_train)
  # Evaluate the logistic regression model
  y_pred_logistic = logistic_model.predict(X_test)
  accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
  print("Logistic Regression Accuracy:", accuracy_logistic)

  # Get the confusion matrix
  cm = confusion_matrix(y_test, y_pred_logistic)

  # Create a heatmap of the confusion matrix
  sns.heatmap(cm, annot=True, fmt="d")
  plt.xlabel("Predicted")
  plt.ylabel("Actual")
  plt.title("Confusion Matrix")
  plt.show()



In [16]:
train_and_evaluate_logisticR(X = reddit_data['clean_comment_stemmed_str'])

ValueError: could not convert string to float: 'and mota bhai did yoga it grave'

In [ ]:
# Train an SVM model
svm_model = SVC()
svm_model.fit(X_train, y_train)

In [ ]:
# Evaluate the SVM model
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)

In [ ]:
# Train a simple neural network model
nn_model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000)
nn_model.fit(X_train, y_train)

In [ ]:
# Evaluate the simple neural network model
y_pred_nn = nn_model.predict(X_test)
accuracy_nn = accuracy_score(y_test, y_pred_nn)
print("Simple Neural Network Accuracy:", accuracy_nn)